In [25]:
import pandas as pd
import numpy as np

In [26]:
ohlcv = pd.read_csv('binance-btc-usdc.csv.gz', index_col='dt')

In [27]:
cdiff = ohlcv.close.pct_change().fillna(0)
dir = pd.Series(0, index=ohlcv.index)
dir[cdiff > .001] = 1
dir[cdiff < -.001] = 2
dir = dir.shift(-1).iloc[:-1].astype(int)
ohlcv = ohlcv.iloc[:-1]

In [28]:
train = ohlcv.loc[:'2020-02-01']
train_dir = dir.loc[:'2020-02-01']
test = ohlcv.loc['2020-02-01':]
test_dir = dir.loc['2020-02-01':]

In [35]:
def hist3(ohlcv):
    ohlcv_h = pd.concat((ohlcv, ohlcv.shift(1).add_prefix('p_'), ohlcv.shift(2).add_prefix('pp_')), axis=1)
    return ohlcv_h

import lightgbm as lgb
gbm = lgb.LGBMClassifier(learning_rate=0.005, n_estimators=200)

from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

tr = FunctionTransformer(hist3)

pl = make_pipeline(tr, gbm)

pl.fit(train, train_dir)
pred = pl.predict_proba(test)

from sklearn.metrics import roc_auc_score
roc_auc_score(test_dir, pred, multi_class='ovo', average='weighted')

0.5602344019170282

In [36]:
signals = pd.Series(pred.argmax(axis=1), index=test.index)
scales = 1+((test.close.shift(-1) - test.close)/test.close).fillna(0)

long = scales[signals==1].prod()
short = (1/scales[signals==2]).prod()

prev_signals = signals.shift(1).fillna(0).astype(int)
ntrades = (signals != prev_signals).sum()
ntrades += ((prev_signals == 2) & (signals == 1)).sum()
ntrades += ((prev_signals == 1) & (signals == 2)).sum()

pd.Series({
    'simple returns': long*short, # long if 1, short if 2 cash if 0, no commissions
    'buy&hold returns': scales.prod(),
    'commission downscale': 0.999**ntrades
})

simple returns          1.597199
buy&hold returns        3.522904
commission downscale    0.003834
dtype: float64